First attempt for PDE done in TensorFlow

Imports

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

#plot settings
plt.close("all")
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = ["Times New Roman"]
plt.rcParams["figure.dpi"] = 100
plt.rcParams["font.size"] = 12

Model Defintition

In [ ]:
def create_model():
    model = {
        "dense1":tf.keras.layers.Dense(50,activation="tanh"), # 50 hidden nuerons 
        "dense2":tf.keras.layers.Dense(50,activation="tanh"),
        "dense3":tf.keras.layers.Dense(50,activation="tanh"),
        "output_layer":tf.keras.layers.Dense(1)
    }
    return model

def call_model(model,x):
    x = model["dense1"](x)
    x = model["dense2"](x)
    x = model["dense3"](x)
    x = model["output_layer"](x)
    return x

PDE function

In [ ]:
def pde(x,model):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x)
        y_pred = call_model(model,x)
        y_x = tape.gradient(y_pred,x)
    y_xx = tape.gradient(y_x,x)
    del tape
    return y_xx + np.pi**2 *tf.sin(np.pi*x)  # residual

Loss function

In [ ]:
def loss(model,x,x_bc,y_bc):
    res = pde(x,model)
    loss_pde = tf.reduce_mean(tf.square(res)) # Compute the MSE of the boundary conditions
    y_bc_pred = call_model(model,x_bc)
    loss_bc = tf.reduce_mean(tf.square(y_bc_pred - y_bc))
    return loss_pde + loss_bc

Training step

In [ ]:
def train_step(model,x,x_bc,y_bc,optimizer):
    with tf.GradientTape() as tape:
        loss_value = loss(model,x,x_bc,y_bc)
    gradients = tape.gradient(loss_value, [layer.trainable_variables for layer in model.values()])
    gradients = [gradients for sublist in gradients for gradients in sublist]  # Flatten the list of gradients
    variables = [var for layer in model.values() for var in layer.trainable_variables]
    optimizer.apply_gradients(zip(gradients, variables))
    return loss_value

Setting up the problem

In [ ]:
# Generate training data
x_train = np.linspace(-1,1,100).reshape(-1,1)
x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)

# Boundary data
x_bc = np.array([[-1], [1]], dtype=np.float32)
y_bc = np.array([[0.0],[0.0]], dtype=np.float32)
x_bc = tf.convert_to_tensor(x_bc, dtype=tf.float32)
y_bc = tf.convert_to_tensor(y_bc, dtype=tf.float32)

# Define the PINN model
model = create_model()

# Define the optimizer with a learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Train the model
epochs = 2000
for epoch in range(epochs):
    lossvalue = train_step(model,x_train,x_bc,y_bc,optimizer)
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss: {lossvalue.numpy()}")
        

Predicting solutions

In [ ]:
x_test = np.linspace(-1, 1, 1000).reshape(-1, 1)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)
y_pred = call_model(model, x_test).numpy()

# Analytical solution
y_true = np.sin(np.pi * x_test)

Plotting results

In [ ]:
plt.figure(figsize=(5, 3))
plt.plot(x_test, y_pred, label='PINN Prediction', color='blue')
plt.plot(x_test, y_true, label='Analytical Solution', linestyle='dashed', color='pink')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.title('PINN vs Analytical Solution')
plt.show()